In [1]:
import pandas as pd
import regex as re
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px
import networkx as nx
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder

c:\Users\alexi\anaconda3\lib\site-packages\xarray\core\pycompat.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion(duck_array_module.__version__)
c:\Users\alexi\anaconda3\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
c:\Users\alexi\anaconda3\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
c:\Users\alexi\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
c:\Users\alexi\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use 

In [2]:
df = pd.read_csv('tel_samp_rec.csv', encoding="ISO-8859-1")
df_theses = df[['disc1.rec.lev1', 'disc2.rec.lev1', 'disc3.rec.lev1']]
df_theses

,disc1.rec.lev1,disc2.rec.lev1,disc3.rec.lev1
0,X,NaN,NaN
1,IV,NaN,NaN
2,X,NaN,NaN
3,V,V,NaN
4,V,V,NaN
...,...,...,...
100041,VIII,NaN,NaN
100042,X,IX,NaN
100043,pharmacie,X,NaN
100044,pharmacie,NaN,NaN


We get rid of any row where all 3 columns are NaN

In [3]:
df_theses.dropna(how='all', inplace=True)
df_theses

c:\Users\alexi\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,disc1.rec.lev1,disc2.rec.lev1,disc3.rec.lev1
0,X,NaN,NaN
1,IV,NaN,NaN
2,X,NaN,NaN
3,V,V,NaN
4,V,V,NaN
...,...,...,...
100040,X,NaN,NaN
100041,VIII,NaN,NaN
100042,X,IX,NaN
100043,pharmacie,X,NaN


In [4]:
fig = px.bar(df['disc1.rec.lev1'].value_counts(), x=df['disc1.rec.lev1'].value_counts().index,y=df['disc1.rec.lev1'].value_counts().values)
fig.update_layout(xaxis_title='Item', yaxis_title='Times Purchased')
fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



We create a merged column where we store a list with the values of the other columns

In [5]:
df_theses['merged'] = [[]] * len(df_theses)
for index, row in df_theses.iterrows():
    df_theses.at[index, 'merged'] = [row['disc1.rec.lev1'], row['disc2.rec.lev1'], row['disc3.rec.lev1']]
df_theses['merged'].head(30)

C:\Users\alexi\AppData\Local\Temp\ipykernel_8448\4023426759.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0        [X, nan, nan]
1       [IV, nan, nan]
2        [X, nan, nan]
3          [V, V, nan]
4          [V, V, nan]
5      [VII, nan, nan]
7       [VI, nan, nan]
8     [VIII, nan, nan]
9      [VII, nan, nan]
10       [V, nan, nan]
11       [V, nan, nan]
12     [III, nan, nan]
13      [IV, nan, nan]
14       [IX, IX, nan]
15      [VI, nan, nan]
16       [V, nan, nan]
17      [IX, nan, nan]
18     [VII, nan, nan]
19       [IX, IX, nan]
20      [IV, nan, nan]
21        [V, II, nan]
22       [V, nan, nan]
23      [VI, nan, nan]
24    [VIII, nan, nan]
25      [VI, nan, nan]
26      [IX, nan, nan]
27      [II, nan, nan]
28      [IX, nan, nan]
29       [V, nan, nan]
30       [V, nan, nan]
Name: merged, dtype: object

We filter the NaN data from the merged column

In [6]:
df_theses['merged'] = [list(filter(lambda x: not pd.isna(x), lst)) for lst in df_theses['merged']]
df_theses['merged'].head(30)

C:\Users\alexi\AppData\Local\Temp\ipykernel_8448\1182878656.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0          [X]
1         [IV]
2          [X]
3       [V, V]
4       [V, V]
5        [VII]
7         [VI]
8       [VIII]
9        [VII]
10         [V]
11         [V]
12       [III]
13        [IV]
14    [IX, IX]
15        [VI]
16         [V]
17        [IX]
18       [VII]
19    [IX, IX]
20        [IV]
21     [V, II]
22         [V]
23        [VI]
24      [VIII]
25        [VI]
26        [IX]
27        [II]
28        [IX]
29         [V]
30         [V]
Name: merged, dtype: object

In [7]:
encoder_thesis = TransactionEncoder().fit(df_theses['merged'])

In [8]:
onehot_thesis = encoder_thesis.transform(df_theses['merged'])
onehot_thesis = pd.DataFrame(onehot_thesis, columns=encoder_thesis.columns_)
onehot_thesis.columns

Index(['I', 'I - Droit', 'II', 'III', 'IV', 'IX', 'V', 'VI', 'VII', 'VIII',
       'X', 'XII', 'pharmacie'],
      dtype='object')

In [9]:
onehot_thesis

,I,I - Droit,II,III,IV,IX,V,VI,VII,VIII,X,XII,pharmacie
0,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96868,False,False,False,False,False,False,False,False,False,False,True,False,False
96869,False,False,False,False,False,False,False,False,False,True,False,False,False
96870,False,False,False,False,False,True,False,False,False,False,True,False,False
96871,False,False,False,False,False,False,False,False,False,False,True,False,True


In [10]:
frequents = apriori(onehot_thesis,min_support=0.001, max_len=10, use_colnames=True )
frequents

,support,itemsets
0,0.005905,(I)
1,0.012687,(I - Droit)
2,0.038277,(II)
3,0.027324,(III)
4,0.090108,(IV)
5,0.281038,(IX)
6,0.202678,(V)
7,0.091078,(VI)
8,0.048342,(VII)
9,0.089468,(VIII)


In [11]:
rules = association_rules(frequents, metric='support', min_threshold=0.001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(I),(IV),0.005905,0.090108,0.001745,0.295455,3.278906,0.001212,1.291460,0.699148
1,(IV),(I),0.090108,0.005905,0.001745,0.019361,3.278906,0.001212,1.013722,0.763849
2,(II),(IV),0.038277,0.090108,0.002416,0.063107,0.700349,-0.001034,0.971180,-0.307905
3,(IV),(II),0.090108,0.038277,0.002416,0.026807,0.700349,-0.001034,0.988214,-0.319835
4,(V),(II),0.202678,0.038277,0.001228,0.006061,0.158344,-0.006529,0.967588,-0.869563
5,(II),(V),0.038277,0.202678,0.001228,0.032093,0.158344,-0.006529,0.823759,-0.846788
6,(IV),(III),0.090108,0.027324,0.002281,0.025318,0.926566,-0.000181,0.997941,-0.080123
7,(III),(IV),0.027324,0.090108,0.002281,0.083491,0.926566,-0.000181,0.992780,-0.075341
8,(III),(XII),0.027324,0.023133,0.001332,0.048734,2.106671,0.000700,1.026913,0.540075
9,(XII),(III),0.023133,0.027324,0.001332,0.057564,2.106671,0.000700,1.032086,0.537758


In [12]:
rules_pruned_support = association_rules(frequents, metric='support', min_threshold=0.002)
rules_pruned_support

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(II),(IV),0.038277,0.090108,0.002416,0.063107,0.700349,-0.001034,0.971180,-0.307905
1,(IV),(II),0.090108,0.038277,0.002416,0.026807,0.700349,-0.001034,0.988214,-0.319835
2,(IV),(III),0.090108,0.027324,0.002281,0.025318,0.926566,-0.000181,0.997941,-0.080123
3,(III),(IV),0.027324,0.090108,0.002281,0.083491,0.926566,-0.000181,0.992780,-0.075341
4,(IV),(XII),0.090108,0.023133,0.003458,0.038378,1.658980,0.001374,1.015853,0.436557
5,(XII),(IV),0.023133,0.090108,0.003458,0.149487,1.658980,0.001374,1.069816,0.406627
6,(V),(IX),0.202678,0.281038,0.011469,0.056586,0.201345,-0.045492,0.762085,-0.832634
7,(IX),(V),0.281038,0.202678,0.011469,0.040808,0.201345,-0.045492,0.831244,-0.846558
8,(IX),(VI),0.281038,0.091078,0.013502,0.048044,0.527505,-0.012094,0.954794,-0.554734
9,(VI),(IX),0.091078,0.281038,0.013502,0.148249,0.527505,-0.012094,0.844099,-0.496342


In [13]:
fig = px.scatter(rules, x="support", y="confidence", color="lift", size="confidence",
                 hover_data=["support", "confidence", "lift", "antecedent support", "consequent support"],
                 hover_name=rules["antecedents"].apply(lambda x: ", ".join(list(x))) + 
                            " -> " + 
                            rules["consequents"].apply(lambda x: ", ".join(list(x))) + 
                            rules["antecedent support"].apply(lambda x: "{:.3f}".format(x)) + 
                            rules["consequent support"].apply(lambda x: "{:.3f}".format(x)))
fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [14]:
fig = px.scatter(rules_pruned_support, x="support", y="confidence", color="lift", size="confidence",
                 hover_data=["support", "confidence", "lift", "antecedent support", "consequent support"],
                 hover_name=rules_pruned_support["antecedents"].apply(lambda x: ", ".join(list(x))) + 
                            " -> " + 
                            rules_pruned_support["consequents"].apply(lambda x: ", ".join(list(x))) + 
                            rules_pruned_support["antecedent support"].apply(lambda x: "{:.3f}".format(x)) + 
                            rules_pruned_support["consequent support"].apply(lambda x: "{:.3f}".format(x)))
fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [15]:
fig = px.parallel_coordinates(rules, dimensions=["antecedent support", "consequent support", "support", "confidence", "lift"],
                            color="lift", color_continuous_scale=px.colors.sequential.Bluyl,
                            title='',
)
fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [16]:
fig = px.parallel_coordinates(rules_pruned_support, dimensions=["antecedent support", "consequent support", "support", "confidence", "lift"],
                            color="lift", color_continuous_scale=px.colors.sequential.Bluyl,
                            title='',
)
fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [17]:
# Create directed graph
G = nx.DiGraph()
for i, row in rules.iterrows():
    G.add_edge(row['antecedents'], row['consequents'], weight=row['confidence'])
    nx.set_node_attributes(G, {row['antecedents']: {'lift': row['lift']}})
    nx.set_node_attributes(G, {row['consequents']: {'lift': row['lift']}})
    nx.set_node_attributes(G, {row['antecedents']: {'support': row['support']}})
    nx.set_node_attributes(G, {row['consequents']: {'support': row['support']}})
# Create plotly figure
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=[[0, 'red'], [1, 'blue']],
        reversescale=True,
        color=[],
        size=[],
        colorbar=dict(
            thickness=15,
            title='Lift',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
pos = nx.spring_layout(G, k=0.15, seed=42) # replace with your preferred layout
nx.set_node_attributes(G, pos, 'pos')

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    text = next(iter(node)) if isinstance(node, frozenset) else node
    text = f"{text}<br>Support: {G.nodes[node]['support']:.4f}<br> Lift: {G.nodes[node]['lift']:.4f}"
    node_trace['text'] += tuple([text])
    node_trace['marker']['color'] += tuple([G.nodes[node]['lift']])
    node_trace['marker']['size'] += tuple([8000*G.nodes[node]['support']])

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Association Rules Graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Market Basket Analysis Retail Data",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Lift",
            thickness=15,
            xanchor="left",
            titleside="right"
        )
    ),
    plot_bgcolor='white'
)

fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [18]:
# Create directed graph
G = nx.DiGraph()
for i, row in rules_pruned_support.iterrows():
    G.add_edge(row['antecedents'], row['consequents'], weight=row['confidence'])
    nx.set_node_attributes(G, {row['antecedents']: {'lift': row['lift']}})
    nx.set_node_attributes(G, {row['consequents']: {'lift': row['lift']}})
    nx.set_node_attributes(G, {row['antecedents']: {'support': row['support']}})
    nx.set_node_attributes(G, {row['consequents']: {'support': row['support']}})
# Create plotly figure
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=[[0, 'red'], [1, 'blue']],
        reversescale=True,
        color=[],
        size=[],
        colorbar=dict(
            thickness=15,
            title='Lift',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
pos = nx.spring_layout(G, k=0.15, seed=42) # replace with your preferred layout
nx.set_node_attributes(G, pos, 'pos')

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    text = next(iter(node)) if isinstance(node, frozenset) else node
    text = f"{text}<br>Support: {G.nodes[node]['support']:.4f}<br> Lift: {G.nodes[node]['lift']:.4f}"
    node_trace['text'] += tuple([text])
    node_trace['marker']['color'] += tuple([G.nodes[node]['lift']])
    node_trace['marker']['size'] += tuple([4000*G.nodes[node]['support']])

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Association Rules Graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Market Basket Analysis Retail Data",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Lift",
            thickness=15,
            xanchor="left",
            titleside="right"
        )
    ),
    plot_bgcolor='white'
)

fig.show()

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\alexi\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

